In [2]:
#!/usr/bin/env python

import sys
import os
absFilePath = os.path.abspath('attractor_switching.py')
fileDir = os.path.dirname(absFilePath)
parentDir = os.path.dirname(fileDir)

# change working directory to the project file of codes, use parentDir else
sys.path.append(parentDir+'/src_and_example')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
from pylab import figure, cm
import functions
import random
seed=50
import cv2


In [13]:
#read data and calculate controlled state

goal='switch'
bifurcation='low'


filename=fileDir+'/data/data_figure_4_5'
with open(filename, 'rb') as pickle_file:
    data = pickle.load(pickle_file)
node_ic=data['node_ic'][0] #take the entry [2] that corresponds to bifurcation low and switchfromto lowtohigh
control=data['controls'][0]

#Set dimensions
T= 600 #simulation and control time 
dt=0.1 #time stepsize
tsteps=int(T/dt) #number of timesteps
N=94
d=2 #dimension of each FitzHugh-Nagumo node
no_control=np.zeros((tsteps,N))
#make dictionary with all parameters
args = {
    'tsteps':tsteps,
    'dt':dt,
    'd':d,
    'noise':0,
    'node_ic':node_ic
    }

parameters = functions.set_parameters(goal,bifurcation,parentDir)

args.update(parameters)

#calculate the states
state_controlled=functions.plot_runge_kutta(functions.ODE_FHN_network,control, **args)

#single uncoupled oscillator
Adj=args['A']
args['A']=np.zeros((args['N'],args['N']))
state_uncoup=functions.plot_runge_kutta(functions.ODE_FHN_network,no_control, **args)
state_uncoup=state_uncoup[:,:,0]
args['A']=Adj

In [17]:
np.min(state_controlled[:,1,:])

0.2710266002794216

In [18]:
fs=30
N=94
degree=np.sum(args['A'],axis=0)
coloring=degree
indices=np.arange(0,6000,3)
cmap='coolwarm'
for i in indices:
    fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(11,7))

    V=np.dstack((control[i],np.zeros(N))).reshape(N,2)*200
    origin=state_controlled[i,:,:]

    ax1.plot(state_uncoup[5000:,0],state_uncoup[5000:,1],color='lightgray',zorder=0)
    Z=ax1.quiver(*origin, V[:,0], V[:,1], scale=1,color='black',zorder=6)
    cax=ax1.scatter(state_controlled[i,0,:],state_controlled[i,1,:],marker='o',c=coloring,s=200,cmap=cmap,zorder=10)
    ax1.quiverkey(Z, 0.8, 0.94, .1, label="$\overline{u} \cdot 200$", labelpos='E',
                  fontproperties={'size': fs-7}, labelcolor='black')

    ax1.set_xlabel('$x_1$',size=fs)
    ax1.set_ylabel('$x_2$',size=fs)
    ax1.set_xlim(-0.35,1.25)
    ax1.set_ylim(.24,95)
    ax1.tick_params(labelsize=fs)
    ax1.grid(True)
    ax1.margins(0) # remove default margins (matplotlib verision 2+)

    cbar=fig.colorbar(cax)
    cbar.set_label('weighted degree $d$',size=fs)
    cbar.ax.tick_params(labelsize=fs)

    if i<5000 and i>1000:
        ax1.set_title('control on \n t='+str(round(i*dt,2))+' timeunits ', y=-.28,x=1,fontsize=fs-4,color='red')
    else:
        ax1.set_title('control off \n t='+str(round(i*dt,2))+' timeunits ', y=-.28,x=1,fontsize=fs-4,color='black')
    
    fig.tight_layout()
    plt.subplots_adjust(right=1,top=1)
    #plt.show()
    plt.savefig(fileDir+"/videos/vid_fig/_"+str(i)+'.png', bbox_inches='tight')
    plt.close()

In [19]:
pathOut = fileDir+"/videos/video_fig4a.avi"
fps = 50

frame_array=[]
for i in indices:
    filename=fileDir+"/videos/vid_fig/_"+str(i)+'.png'
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()